# Covid-19 Death rate
In this work I want to see if there is any relatonship between a state feathers and the death rate from coronavirus.
## Data prepration
### Coronavirus data

In [58]:
import pandas as pd
from datetime import date,timedelta
last_updated_date = date.today()
delta = timedelta(days=1)
import requests
base_url =  'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/'
url = base_url+last_updated_date.strftime("%m-%d-%Y")+'.csv'
while requests.get(url).status_code==404:
    last_updated_date-=delta
    url = base_url+last_updated_date.strftime("%m-%d-%Y")+'.csv'
print("last updated file found in %s"%last_updated_date)
    
df = pd.read_csv(url,index_col=0,parse_dates=[0])

last updated file found in 2020-03-26


In [78]:
df_usa_raw = df[df["Country_Region"]=="US"]

In [102]:
df_usa=df_usa_raw.drop(columns=['Admin2','Country_Region', 'Last_Update', 'Lat','Long_'])
df_usa_states_raw = df_usa.groupby('Province_State').sum()
df_usa_states_raw=df_usa_states_raw.sort_values('Province_State')

In [112]:
df_usa_states =df_usa_states_raw.drop(index=['Grand Princess','Diamond Princess','District of Columbia','Northern Mariana Islands',\
                                             'Recovered','American Samoa','Virgin Islands','Guam','Puerto Rico','Virgin Islands'])

In [113]:
len(df_usa_states)

50